In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import gpxpy
#import gpx.gpx
import geopy.distance
import folium

In [19]:
def Analyze_GPX(filename='test_track.gpx', maxlean=40, accel=4, topspeed='180', delta=3,full=True):
    # import gps coordinates for route from gpx file exported from google maps & converter
    gpx = gpxpy.parse(open(filename,'r'))
    points = []
    for track in gpx.tracks:
        for segment in track.segments:        
            for point in segment.points:
                points.append([point.latitude, point.longitude])
    points = np.array(points)

    curvature = np.zeros_like(points[:,0])

    for i in range(delta,len(points)-2*delta-1):
        a = math.sqrt(np.sum((points[i-delta] - points[i])**2))
        b = math.sqrt(np.sum((points[i] - points[i+delta])**2))
        c = math.sqrt(np.sum((points[i-delta] - points[i+delta])**2))
        if a*b*c > 0 and a+b-c>0:
            cosangle = (a**2 + b**2 - c**2) /(2*a*b)
            try:
                sinangle = math.sqrt(1 - cosangle**2)
                radius = c / sinangle
            except: pass
        scale = geopy.distance.distance(points[i], points[i+delta]).m / b
        curvature[i] = radius * scale

    # calculate theoretical top speed for each stretch of road based on max lean angle and acceleration
    topspeed *= 0.44704
    max_vel = np.zeros_like(curvature)

    cornering_accel = 9.81 * np.tan(maxlean * np.pi / 180)
    r_min = cornering_accel**2/topspeed**2

    for i in range(len(curvature)):
        max_vel[i] = cornering_accel * np.sqrt(curvature[i])/2
    max_vel[max_vel > topspeed] = topspeed
    max_vel[max_vel < 3] = 3

    # iterate simulation to determine actual top speed for each stretch of road
    braking_dist = topspeed**2 / (2*accel)
    vel = np.ones_like(max_vel)

    # calculate distances between points
    next_dist = np.zeros_like(max_vel)
    for i in range(len(points)-1):
        next_dist[i] = geopy.distance.distance(points[i], points[i+1]).m

    for i in range(len(points)-1):
        # determine how far ahead to look on route and determine info
        point_search = 0
        cum_dist = 0
        while cum_dist < braking_dist and i+point_search < len(points):
            cum_dist += next_dist[i + point_search]
            point_search += 1
        max_vel_distro = [max_vel[i+j] for j in range(point_search)]
        dist_distro = np.cumsum([next_dist[i+j] for j in range(point_search)])
        
        # determine action (brake or accelerate)
        future_action = np.zeros_like(dist_distro)
        BRAKE = False
        for j in range(len(max_vel_distro)):
            if dist_distro[j] == 0:
                future_action[j] = 0
            else:
                future_action[j] = (max_vel_distro[j]**2 - vel[i]**2) / (2*dist_distro[j])
            
            if future_action[j] <= -0.9*accel:
                BRAKE = True
        if BRAKE:
            vel[i+1] = vel[i] - accel
        elif future_action[0] > accel:
                vel[i+1] = vel[i] + accel
        else:
            vel[i+1] = vel[i] + future_action[0]
        
        # limit speed to topspeed
        if vel[i+1] >= topspeed:
            vel[i+1] = topspeed
    
    if full:
        print('Input Parameters:\n   ', round(topspeed*2.24), 'mph top speed\n   ',
            round(accel), 'm/s² acceleration\n   ',
            round(maxlean), '° max lean\n   ',
            delta, 'point delta')
        print('Distance:', np.round(np.sum(next_dist)/1609.34,3),'mi')

        sec = np.sum(next_dist/vel)
        print('Time:', int(sec//60//60),':', int(sec//60%60),':', np.round(sec%60, 2))
        print('Speed: Avg.', np.round(np.sum(next_dist)/sec*2.24,2), '  Max:', np.round(max(vel)*2.2))
        print(len(points), 'GPS points on route')

        l = 0
        h = -1


        plt.figure(figsize=(15,7)) # set size of figure
        plt.scatter(points[l:h,1], points[l:h,0], c=vel[l:h]*2.24, s=3, cmap='magma')
        plt.colorbar();
    else:
        print('Distance:', np.round(np.sum(next_dist)/1609.34,3),'mi')
        sec = np.sum(next_dist/vel)
        print('Speed: Avg.', np.round(np.sum(next_dist)/sec*2.24,2))

In [20]:
#maxlean, accel, topspeed = 35, 3, 150
#maxlean, accel, topspeed = 40, 4, 170
#maxlean, accel, topspeed = 45, 5, 190

gpx_list = ['segment'+str(i)+'.gpx' for i in range(0,28)]

for seg in gpx_list:
    print(seg)
    Analyze_GPX(seg, 45, 4, 180, full=False)
    Analyze_GPX(seg, 35, 4, 180, full=False)
    print('\n')

segment0.gpx
Distance: 95.427 mi
Speed: Avg. 93.08
Distance: 95.427 mi
Speed: Avg. 162.23


segment1.gpx
Distance: 95.305 mi
Speed: Avg. 106.17
Distance: 95.305 mi
Speed: Avg. 171.09


segment2.gpx
Distance: 96.248 mi
Speed: Avg. 95.2
Distance: 96.248 mi
Speed: Avg. 165.21


segment3.gpx
Distance: 96.203 mi
Speed: Avg. 84.66
Distance: 96.203 mi
Speed: Avg. 164.8


segment4.gpx
Distance: 96.361 mi
Speed: Avg. 92.51
Distance: 96.361 mi
Speed: Avg. 172.86


segment5.gpx
Distance: 96.493 mi
Speed: Avg. 122.11
Distance: 96.493 mi
Speed: Avg. 174.22


segment6.gpx
Distance: 96.601 mi
Speed: Avg. 120.03
Distance: 96.601 mi
Speed: Avg. 176.24


segment7.gpx
Distance: 99.094 mi
Speed: Avg. 107.44
Distance: 99.094 mi
Speed: Avg. 167.18


segment8.gpx
Distance: 97.338 mi
Speed: Avg. 117.4
Distance: 97.338 mi
Speed: Avg. 175.19


segment9.gpx
Distance: 98.029 mi
Speed: Avg. 85.35
Distance: 98.029 mi
Speed: Avg. 127.24


segment10.gpx
Distance: 98.706 mi
Speed: Avg. 99.55
Distance: 98.706 mi
Speed: